In [2]:
import os
import math
import numpy as np
import numpy.linalg as npla
import scipy
import scipy.sparse.linalg as spla
from scipy import sparse
from scipy import linalg
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
# %matplotlib tk

In [2]:
from scipy import sparse

def make_A_3D(k):
    """
    Create the matrix for the temperature problem on a k-by-k-by-k 3D grid.
    Parameters:
        k: number of grid points in each dimension.
    Outputs:
        A: the sparse k**3-by-k**3 matrix representing the finite difference approximation to Poisson's equation.
    """
    triples = []
    for l in range(k): #level
        for r in range(k): #row
            for c in range(k): #col
                row = c + r*k + l*k*k
                triples.append((row,row, 6.0))
                #connect to left
                if c>0:
                    triples.append((row,row-1,-1.0))
                #connect to right
                if c<k-1:
                    triples.append((row,row+1,-1.0))
                #connect to last row
                if r>0:
                    triples.append((row,row-k,-1.0))
                #connect to next row
                if r<k-1:
                    triples.append((row,row+k,-1.0))
                #connect to last level
                if l>0:
                    triples.append((row,row-k*k,-1.0))
                if l<k-1:
                    triples.append((row,row+k*k,-1.0))
    #convert list of triples to a scipy sparse matrix
    ndim = k*k*k
    rownum = [t[0] for t in triples]
    colnum = [t[1] for t in triples]
    values = [t[2] for t in triples]
    A = sparse.csr_matrix((values,(rownum, colnum)), shape = (ndim, ndim))
    
    return A


In [3]:
def LUfactor(A, pivoting = True):
    """Factor a square matrix with partial pivoting, A[p,:] == L @ U
    Parameters: 
      A: the matrix.
      pivoting = True: whether or not to do partial pivoting
    Outputs (in order):
      L: the lower triangular factor, same dimensions as A, with ones on the diagonal
      U: the upper triangular factor, same dimensions as A
      p: the permutation vector that permutes the rows of A by partial pivoting
    """
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # Initialize p to be the identity permutation
    p = np.array(range(n))
    
    # Make a copy of the matrix that we will transform into L and U
    LU = A.astype(np.float64).copy()
    
    # Eliminate each column in turn
    for piv_col in range(n):
        
        # Choose the pivot row and swap it into place
        if pivoting:
            piv_row = piv_col + np.argmax(np.abs(LU[piv_col:, piv_col]))
            assert LU[piv_row, piv_col] != 0., "can't find nonzero pivot, matrix is singular"
            LU[[piv_col, piv_row], :]  = LU[[piv_row, piv_col], :]
            p[[piv_col, piv_row]]      = p[[piv_row, piv_col]]
            
        # Update the rest of the matrix
        pivot = LU[piv_col, piv_col]
        assert pivot != 0., "pivot is zero, can't continue"
        for row in range(piv_col + 1, n):
            multiplier = LU[row, piv_col] / pivot
            LU[row, piv_col] = multiplier
            LU[row, (piv_col+1):] -= multiplier * LU[piv_col, (piv_col+1):]
            
    # Separate L and U in the result
    U = np.triu(LU)
    L = LU - U + np.eye(n)
    
    return (L, U, p)

In [4]:
def Lsolve(L, b, unit_diag = False):
    """Forward solve a unit lower triangular system Ly = b for y
    Parameters: 
      L: the matrix, must be square, lower triangular, with ones on the diagonal
      b: the right-hand side vector
        unit_diag = False: if true, assume the diagonal is all ones
    Output:
      y: the solution vector to L @ y == b
    """
    # Check the input
    m, n = L.shape
    assert m == n, "matrix must be square"
    assert np.all(np.tril(L) == L), "matrix L must be lower triangular"
    if unit_diag:
        assert np.all(np.diag(L) == 1), "matrix L must have ones on the diagonal"
    bn, = b.shape
    assert bn == n, "rhs vector must be same size as L"

    # Make a copy of b that we will transform into the solution
    y = b.astype(np.float64).copy()
    
    # Forward solve
    for col in range(n):
        if not unit_diag:
            y[col] /= L[col, col]
        y[col+1:] -= y[col] * L[col+1:, col]
        
    return y

In [5]:
def Usolve(U, y, unit_diag = False):
    """Backward solve an upper triangular system Ux = y for x
    Parameters: 
      U: the matrix, must be square, upper triangular, with nonzeros on the diagonal
      y: the right-hand side vector
      unit_diag = False: if true, assume the diagonal is all ones
    Output:
      x: the solution vector to U @ x == y
    """
    # Check the input
    m, n = U.shape
    assert m == n, "matrix must be square"
    assert np.all(np.triu(U) == U), "matrix U must be upper triangular"
    if unit_diag:
        assert np.all(np.diag(U) == 1), "matrix U must have ones on the diagonal"
    yn, = y.shape
    assert yn == n, "rhs vector must be same size as U"
    
    # Make a copy of y that we will transform into the solution
    x = y.astype(np.float64).copy()
    
    # Back solve
    for col in reversed(range(n)):
        if not unit_diag:
            x[col] /= U[col, col]
        x[:col] -= x[col] * U[:col, col]
        
    return x

In [6]:
def LUsolve(A, b):
    """Solve a linear system Ax = b for x by LU factorization with partial pivoting.
    Parameters: 
      A: the matrix.
      b: the right-hand side
    Outputs (in order):
      x: the computed solution
      rel_res: relative residual norm,
        norm(b - Ax) / norm(b)
    """
    # Check the input
    m, n = A.shape
    assert m == n, "matrix must be square"
    bn, = b.shape
    assert bn == n, "rhs vector must be same size as matrix"
    
    # LU factorization
    L, U, p = LUfactor(A)
    
    # Forward and back substitution
    y = Lsolve(L,b[p])
    x = Usolve(U,y)
    
    # Residual norm
    rel_res = npla.norm(b - A@x) / npla.norm(b)
    
    return (x, rel_res)

In [7]:
def Jsolve(A, b, tol = 1e-8, max_iters = 1000, callback = None):
    """Solve a linear system Ax = b for x by the Jacobi iterative method.
    Parameters: 
      A: the matrix.
      b: the right-hand side vector.
      tol = 1e-8: the relative residual at which to stop iterating.
      max_iters = 1000: the maximum number of iterations to do. 
      callback = None: a user function to call at every iteration. 
        The callback function has arguments 'x', 'iteration', and 'residual'
    Outputs (in order):
      x: the computed solution
      rel_res: list of relative residual norms at each iteration.
        The number of iterations actually done is len(rel_res) - 1
    """
    # Check the input
    m, n = A.shape
    assert m == n, "matrix must be square"
    bn, = b.shape
    assert bn == n, "rhs vector must be same size as matrix"

    # Split A into diagonal D plus off-diagonal C
    d = A.diagonal()         # diagonal elements of A as a vector
    C = A.copy()             # copy of A ...
    C.setdiag(np.zeros(n))   # ... without the diagonal
    
    # Initial guess: x = 0
    x = np.zeros(n)

    # Vector of relative residuals
    # Relative residual is norm(residual)/norm(b)
    # Intitial residual is b - Ax for x=0, or b
    rel_res = [1.0]
        
    # Call user function if specified
    if callback is not None:
        callback(x = x, iteration = 0, residual = 1)

    # Iterate
    for k in range(1, max_iters+1):
        # New x
        x = (b - C @ x) / d

        # Record relative residual
        this_rel_res = npla.norm(b - A @ x) / npla.norm(b)
        rel_res.append(this_rel_res)
                
        # Call user function if specified
        if callback is not None:
            callback(x = x, iteration = k, residual = this_rel_res)
                        
        # Stop if within tolerance    
        if this_rel_res <= tol:
            break
            
    return (x, rel_res)

In [8]:
def make_A(k):
    """Create the matrix for the temperature problem on a k-by-k grid.
    Parameters: 
      k: number of grid points in each dimension.
    Outputs:
      A: the sparse k**2-by-k**2 matrix representing the finite difference approximation to Poisson's equation.
    """
    # First make a list with one triple (row, column, value) for each nonzero element of A
    triples = []
    for i in range(k):
        for j in range(k):
            # what row of the matrix is grid point (i,j)?
            row = j + i*k
            # the diagonal element in this row
            triples.append((row, row, 4.0))
            # connect to left grid neighbor
            if j > 0:
                triples.append((row, row - 1, -1.0))
            # ... right neighbor
            if j < k - 1:
                triples.append((row, row + 1, -1.0))
            # ... neighbor above
            if i > 0:
                triples.append((row, row - k, -1.0))
            # ... neighbor below
            if i < k - 1:
                triples.append((row, row + k, -1.0))
    
    # Finally convert the list of triples to a scipy sparse matrix
    ndim = k*k
    rownum = [t[0] for t in triples]
    colnum = [t[1] for t in triples]
    values = [t[2] for t in triples]
    A = sparse.csr_matrix((values, (rownum, colnum)), shape = (ndim, ndim))
    
    return A

In [9]:
def CGsolve(A, b, tol = 1e-8, max_iters = 1000, callback = None):
    """Solve a linear system Ax = b for x by the conjugate gradient iterative method.
    Parameters: 
      A: the matrix.
      b: the right-hand side vector.
      tol = 1e-8: the relative residual at which to stop iterating.
      max_iters = 1000: the maximum number of iterations to do. 
      callback = None: a user function to call at every iteration, with one argument x
    Outputs (in order):
      x: the computed solution
      rel_res: list of relative residual norms at each iteration.
        The number of iterations actually done is len(rel_res) - 1
    """
    # Check the input
    m, n = A.shape
    assert m == n, "matrix must be square"
    bn, = b.shape
    assert bn == n, "rhs vector must be same size as matrix"

    # Initial guess: x = 0
    x = np.zeros(n)
    
    # Initial residual: r = b - A@0 = b
    r = b
 
    # Initial step is in direction of residual.
    d = r

    # Squared norm of residual
    rtr = r.T @ r
    
    # Vector of relative residuals
    # Relative residual is norm(residual)/norm(b)
    # Intitial residual is b - Ax for x=0, or b
    rel_res = [1.0]
    
    # Call user function if specified
    if callback is not None:
        callback(x = x, iteration = 0, residual = 1)

    # Iterate
    for k in range(1, max_iters+1):
        Ad = A @ d
        alpha = rtr / (d.T @ Ad)  # Length of step
        x = x + alpha * d         # Update x to new x
        r = r - alpha * Ad        # Update r to new residual
        rtrold = rtr 
        rtr = r.T @ r
        beta = rtr / rtrold    
        d = r + beta * d          # Update d to new step direction
                   
        # Record relative residual
        this_rel_res = npla.norm(b - A @ x) / npla.norm(b)
        rel_res.append(this_rel_res)
                
        # Call user function if specified
        if callback is not None:
            callback(x = x, iteration = k, residual = this_rel_res)
                        
        # Stop if within tolerance    
        if this_rel_res <= tol:
            break
            
    return (x, rel_res)

In [129]:
k = 5
b = 10*np.round(np.random.rand(k*k*k))
A = make_A_3D(k)
print(A.todense())

[[ 6. -1.  0. ...  0.  0.  0.]
 [-1.  6. -1. ...  0.  0.  0.]
 [ 0. -1.  6. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  6. -1.  0.]
 [ 0.  0.  0. ... -1.  6. -1.]
 [ 0.  0.  0. ...  0. -1.  6.]]


In [130]:
%%time
res = []
def collect_residual(x):
    res.append(npla.norm(A @ x - b) / npla.norm(b))
x, n_iter = spla.cg(A, b, tol=1e-12,callback=collect_residual)
print("cg residuals for each iteration: ")
#for i in range(0, len(res), 100):
#for i in range(len(res)):
    #print("Iter", i+1, "rel_res:", res[i])

cg residuals for each iteration: 
CPU times: user 3.28 ms, sys: 1.5 ms, total: 4.78 ms
Wall time: 3.56 ms


In [131]:
%%time
X, res = Jsolve(A, b, tol = 1e-12, max_iters = 10000, callback = None)
#for i in range(0, len(res), 100):
#for i in range(len(res)):
    #print("Iter", i+1, "rel_res:", res[i])

CPU times: user 7.3 ms, sys: 2.4 ms, total: 9.69 ms
Wall time: 7.7 ms


In [132]:
%%time
X, rel_res = CGsolve(A, b, tol = 1e-12, max_iters = 400, callback = None)
#print("x =", Xcg[0])
#for i in range(0, len(res), 100):
#for i in range(len(rel_res)):
    #print("Iter", i+1, "rel_res:", rel_res[i])

CPU times: user 2.28 ms, sys: 1.1 ms, total: 3.38 ms
Wall time: 2.32 ms


In [133]:
%%time
x = spla.spsolve(A, b)
res = npla.norm(A @ x - b) / npla.norm(b)
#print("SPsolve residual :", res)

CPU times: user 1.32 ms, sys: 1.1 ms, total: 2.42 ms
Wall time: 1.12 ms


In [134]:
%%time
x, res = LUsolve(np.array(A.todense()), b)
#print("LUsolve rel_res :", res)

CPU times: user 41.2 ms, sys: 4.22 ms, total: 45.4 ms
Wall time: 42.5 ms


In [10]:
A = np.array([[1,3,2],[5,4,6],[6,2,7]])

In [11]:
L,U,p = LUfactor(A)

In [13]:
P = np.eye(A.shape[0])

In [15]:
P = P[p]

In [17]:
P@A

array([[6., 2., 7.],
       [1., 3., 2.],
       [5., 4., 6.]])

In [18]:
A[p]

array([[6, 2, 7],
       [1, 3, 2],
       [5, 4, 6]])

In [19]:
print(P@A - L@U)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [21]:
P = np.array([[0.,1.,0.,0.,0.],
 [0. ,0. ,0. ,1. ,0.],
 [0. ,0. ,0. ,0. ,1.],
 [0. ,0. ,1. ,0. ,0.],
 [1. ,0. ,0. ,0. ,0.]])

In [25]:
A = np.round(10*np.random.rand(5,5))

In [26]:
print(A)

[[10.  6. 10.  8.  9.]
 [ 7.  3.  4.  6.  7.]
 [ 7.  1.  6.  9.  6.]
 [ 2.  6.  7.  8.  7.]
 [ 6.  6.  5.  1.  3.]]


In [27]:
A@P

array([[ 9., 10.,  8.,  6., 10.],
       [ 7.,  7.,  6.,  3.,  4.],
       [ 6.,  7.,  9.,  1.,  6.],
       [ 7.,  2.,  8.,  6.,  7.],
       [ 3.,  6.,  1.,  6.,  5.]])

In [28]:
p = np.array([4,0,3,1,2])

In [29]:
print(A[:,p])

[[ 9. 10.  8.  6. 10.]
 [ 7.  7.  6.  3.  4.]
 [ 6.  7.  9.  1.  6.]
 [ 7.  2.  8.  6.  7.]
 [ 3.  6.  1.  6.  5.]]


In [30]:
A = np.array([[1,2],[3,4]])
Q, R = npla.qr(A)
print("Q:\n", Q,"\nR:\n", R, "\nQ@R:\n", Q@R)

Q:
 [[-0.31622777 -0.9486833 ]
 [-0.9486833   0.31622777]] 
R:
 [[-3.16227766 -4.42718872]
 [ 0.         -0.63245553]] 
Q@R:
 [[1. 2.]
 [3. 4.]]


In [34]:
A = np.array([[1,2,4],[0,0,5],[0,3,6]])
Q, R = npla.qr(A, mode='reduced')
print("Q:\n", Q,"\nR:\n", R, "\nQ@R:\n", Q@R)

Q:
 [[ 1.  0.  0.]
 [-0.  0. -1.]
 [-0. -1.  0.]] 
R:
 [[ 1.  2.  4.]
 [ 0. -3. -6.]
 [ 0.  0. -5.]] 
Q@R:
 [[1. 2. 4.]
 [0. 0. 5.]
 [0. 3. 6.]]


In [35]:
A = np.array([[1,1000],[0,1]])
b1 = np.array([0,1])
x1 = npla.solve(A,b1)

x2 = np.array([-999.999, 1.001])

print("x1:", x1, "\nx2:", x2)
print("\nA @ x1:", A@x1)
print("A @ x2:", A@x2)
print("\nresidual norm:", npla.norm(b1 - A@x2))
print("\ncondition number of A:\n",npla.cond(A))

x1: [-1000.     1.] 
x2: [-999.999    1.001]

A @ x1: [0. 1.]
A @ x2: [1.001 1.001]

residual norm: 1.0010004995002375

condition number of A:
 1000001.9999990001


In [36]:
A = np.array([[1,9],[9,1]])
b1 = np.array([9,1])
x1 = npla.solve(A,b1)

x2 = np.array([0.001, 0.999])

print("x1:", x1, "\nx2:", x2)
print("\nA @ x1:", A@x1)
print("A @ x2:", A@x2)
print("\nresidual norm:", npla.norm(b1 - A@x2))
print("\ncondition number of A:\n",npla.cond(A))

x1: [0. 1.] 
x2: [0.001 0.999]

A @ x1: [9. 1.]
A @ x2: [8.992 1.008]

residual norm: 0.011313708498985399

condition number of A:
 1.2499999999999998


In [37]:
A = np.array([[1,1,0],[1,0,1],[0,1,1]])
b1 = np.array([1,2,1])
x1 = npla.solve(A,b1)

x2 = np.array([1.1,0.1,1])

print("x1:", x1, "\nx2:", x2)
print("\nA @ x1:", A@x1)
print("A @ x2:", A@x2)
print("\nresidual norm:", npla.norm(b1 - A@x2))
print("\ncondition number of A:\n",npla.cond(A))

x1: [ 1. -0.  1.] 
x2: [1.1 0.1 1. ]

A @ x1: [1. 2. 1.]
A @ x2: [1.2 2.1 1.1]

residual norm: 0.24494897427831802

condition number of A:
 2.0000000000000004


In [50]:
A = np.array([[1,1,1],[1,2,3],[1,3,6]])
x = npla.solve(A,[3,8,15])
y = A @ [3,-1,0]
L,U,p = LUfactor(A)
B = linalg.cholesky(A,lower = True)
C = linalg.inv(A)

In [79]:
A = np.random.rand(12,5)
Q,R=linalg.qr(A)
print(Q.shape,R.shape)

(12, 12) (12, 5)


In [76]:
a,b = Q.shape
print(a,b)
b,c = R.shape
print(b,c)
b = np.array(range(12))

12 12
12 5


In [4]:
A = np.array([[1,0,0],[1,1,0],[1,1,1]])
A

array([[1, 0, 0],
       [1, 1, 0],
       [1, 1, 1]])

In [7]:
np.sum(A,0)

array([3, 2, 1])

In [8]:
1023-53

970